## Modeling of Smart Grid Control Center Operations: The Operators Standpoint

In [ ]:
# importing relevant libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import PIL.Image
import time
import IPython.display
from copy import copy
from scipy.integrate import odeint
from io import BytesIO as bio
from abc import ABC, abstractmethod
%matplotlib inline

In [ ]:
# define a Game class characterized by a set of rules, an initial state and the grid size
class Game:
    def __init__(self, initial_state, rules, max_size):
        self.initial_state = initial_state
        self.rules = rules
        self.max_size = max_size
    def run_game(self, iteration):
        state = self.initial_state
        previous_state = None
        progression = []
        i = 0
        while (not state.equals(previous_state) and i < iteration):
            i += 1
            previous_state = state.copy()
            progression.append(previous_state.grid)
            state = state.apply_rules(self.rules,self.max_size)
        progression.append(state.grid)
        return progression

In [ ]:
# define an abstract class that defines methods that will be created within all classes built from the SparseRepState class
class State(ABC):
    @abstractmethod
    def copy(self):
        pass

    @abstractmethod
    def apply_rules(self, rules, max_size):
        pass

    @abstractmethod
    def equals(self, other):
        pass

    @abstractmethod
    def get_neighbours(self, elem, max_size):
        pass

In [ ]:
# define states through a Sparse Representation class that encodes only the subjects of the sparse representation
class SparseRepState(State):
    def __init__(self, grid):
        self.grid = grid

    def copy(self):
        return SparseRepState(copy(self.grid))
    
    def get_neighbours(self, elem, max_size):
        #Returns the neighbours of a live cell if they lie within the bounds of the grid specified by max_size
        l = []
        if elem[0]-1 >= 0:
            l.append((elem[0]-1, elem[1]))
        if elem[0]-1 >= 0 and elem[1]-1 >= 0:
            l.append((elem[0]-1, elem[1]-1))
        if elem[0]-1 >= 0 and elem[1]+1 < max_size:
            l.append((elem[0]-1, elem[1]+1))
        if elem[1]-1 >= 0:
            l.append((elem[0], elem[1]-1))
        if elem[1]-1 >= 0 and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]-1))
        if elem[1]+1 < max_size:
            l.append((elem[0], elem[1]+1))
        if elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]))
        if elem[1]+1 < max_size and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]+1))
        return l

    def equals(self, other):
        if other is None:
            return False
        return self.grid == other.grid

    def apply_rules(self, rules, max_size):
        #Calls the actual rules and provides them with the grid and the neighbour function
        self.grid = rules.apply_rules(self.grid, max_size,self.get_neighbours)
        return self

In [ ]:
# define an abstract class that defines methods that will be created within all classes built from the SparseRepRules class
class Rule(ABC):
    @abstractmethod
    def apply_rules(self, grid, max_size, get_neighbours):
        pass

# Cascading Phase Modeling

#### Here, I make an attempt to simulate what happens when signals are sparsely distributed in a decision making center - as in the case of simultaneous alarms (cognitive overloads).

The following are a set of rules of engagement in this scenario. A cell becomes activated - becomes a signal if at least 3 of its neighbours are already existing signals. A remains a signal if 2 or 3 of its neighbors are signals. Otherwise, it is a noise - normal operating condition.

In [ ]:
# define rules of the game that guides the states evolution
class SparseRepRules(Rule):
    def apply_rules(self, grid, max_size, get_neighbours):
        #grid = state.grid
        counter = {}
        for elem in grid:
            if elem not in counter:
                counter[elem]=0
            nb = get_neighbours(elem, max_size)
            for n in nb:
                if n not in counter:
                    counter[n] = 1
                else:
                    counter[n] += 1
        for c in counter:
            if (counter[c] < 2 or  counter[c] > 3):
                grid.discard(c)
            if counter[c] == 3:
                grid.add(c)
        return grid

The variables below are needed for this simulation. Here, I randomly encode signals (white/dead cells) across the grid. The goal is to show what happens in the center when simultaneous alarms happen and there's a slow intervention by operators. This can potentially culminate into the cascading phase of a blackout.


In [ ]:
# defining grid size, number of iterations, configurations 
num_iter = 1500
grid_length = 80

init = np.zeros((grid_length, grid_length), dtype = bool)
board = {(39, 40),(39, 41),(40, 39),(40, 40),(41, 40)}

rules = SparseRepRules()
game = Game(SparseRepState(board), rules, grid_length)

run_time = time.time()
game_run = game.run_game(num_iter)

print(time.time()-run_time)

In [ ]:
#transform sparse representation to an array that can be plotted
res = np.zeros((len(game_run), grid_length, grid_length), dtype=bool)
for l in range(0,len(game_run)):
    for key in game_run[l]:
        res[l,key[0], key[1]] = True

In [ ]:
#transform array to a gif and save to a file
def save_gif(array, file_name):
    array = np.uint8(np.clip(array,0,1)*255.0)
    frames = []
    for frame in range(array.shape[0]):
        img = PIL.Image.fromarray(array[frame])
        img = img.resize((500, 500))
        frames.append(img)
    img.save(file_name, save_all=True, duration=33.33, append_images=frames, loop=0,size=(500,500))
    
save_gif(res,"output1.gif")

# Characterizing Cascading Signals

For the Smart Grid Decision Making Center, I model potential cascading signals with a sigmoid function characterized by a Logistic Differential Equation.<br/>

\begin{equation*}
\frac{dN}{dt} = rN(1 - \frac{N}{K})
\end{equation*}

where <br/>
r = Cascading Growth Rate until asymptosis <br/>
K = Maximum allowable signal magnitude given the area of coverage of the distribution center <br/>
N = Signal Magnitude at current model run time represented by the number of tripped components <br/>

The differential represents the change in cascading rate with respect to the change in detection time

In [ ]:
def logistic(n, t, r):
    k = 10
    dndt = r*n*(1-n/k)
    return dndt

# initial condition
n_init = 1

# time points
t = np.linspace(0, 60)  

# solving logistic differential equations for different cascading growth rate with odeint
r = 0.2
n1 = odeint(logistic, n_init, t, args = (r,))
r = 0.4
n2 = odeint(logistic, n_init, t, args = (r,))
r = 0.5
n3 = odeint(logistic, n_init, t, args = (r,))

# plot results
plt.plot(t, n1, "r-", linewidth = 2, label = "k = 0.2")
plt.plot(t, n2, "b--", linewidth = 2, label = "k = 0.4")
plt.plot(t, n3, "g:", linewidth = 2, label = "k = 0.5")

# plot label and title
plt.xlabel("Time (minutes)")
plt.ylabel("Signal Magnitude")
plt.title("Plot of Signal Magnitude against Time")

# plot aesthetics
plt.grid(b=True, which='major', color='#666666', linestyle='-', alpha = 0.6)
plt.minorticks_on()
plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
plt.legend(loc = "best")

# display plot
plt.show()

# Agent - Agent Interactions

Here, I simulate agent-agent interactions. I assume that operators can either be distracted (white cells) or focused (black cells). A cell remains focused if less than 4 of its neighboring operators are distracted. Otherwise, the operator becomes distracted.

In [ ]:
# define rules of the game that guides the states evolution
class SparseRepRules(Rule):
    def apply_rules(self, grid, max_size, get_neighbours):
        #grid = state.grid
        counter = {}
        for elem in grid:
            if elem not in counter:
                counter[elem]=0
            nb = get_neighbours(elem, max_size)
            for n in nb:
                if n not in counter:
                    counter[n] = 1
                else:
                    counter[n] += 1
        for c in counter:
            if (counter[c] < 2 or  counter[c] > 3):
                grid.discard(c)
            if counter[c] == 3:
                grid.add(c)
        return grid  

The variables below are needed for this simulation. Here, I randomly encode signals (white/dead cells) across the grid.


In [ ]:
# defining grid size, number of iterations, configurations 
num_iter = 1500
grid_length = 200

init = np.zeros((grid_length, grid_length), dtype = bool)

# sparsely distribute 36 distracted operators across the grid
board = {(50,180), (51,180), (50,181), (51,181), (60,180), (60,179), (60,181), (61,178),
         (62,177), (63,177), (61,182), (62,183), (63,183), (65,182), (66,181), (66,180),
         (66,179), (65,178), (64,180), (67,180), (70,181), (70,182), (70,183), (71,181),
         (71,182), (71,183), (72,180), (72,184), (74,180), (74,179), (74,184), (74,185),
         (84,182), (84,183), (85,182), (85,183)} 

rules = SparseRepRules()
game = Game(SparseRepState(board), rules, grid_length)

run_time = time.time()
game_run = game.run_game(num_iter)

print(time.time()-run_time)

In [ ]:
#transform sparse representation to an array that can be plotted
ress = np.zeros((len(game_run), grid_length, grid_length), dtype=bool)
for l in range(0,len(game_run)):
    for key in game_run[l]:
        ress[l,key[0], key[1]] = True

In [ ]:
#transform array to a gif and save to a file
def save_gif(array, file_name):
    array = np.uint8(np.clip(array,0,1)*255.0)
    frames = []
    for frame in range(array.shape[0]):
        img = PIL.Image.fromarray(array[frame])
        img = img.resize((500, 500))
        frames.append(img)
    img.save(file_name, save_all=True, duration=33.33, append_images=frames, loop=0,size=(500,500))
    
save_gif(ress,"output2.gif")